## Generate Stalk windows
#### Upload gromacs chain coordinate!
===============================================================================================

In [2]:
import numpy as np
import MDAnalysis as md
from MDAnalysis.analysis import align
import subprocess
import os
from MDAnalysis.analysis.leaflet import LeafletFinder

In [3]:
## Define paths to forcefield files and mdp files
ff_loc  = 'ff_itps/'
mdp_loc = 'mdps/'

## Define path to current directory
current_dir = '.'

## Define path to the directory where the stacked system was built
build_dir = '..'

In [6]:
def create_window(umbrella_window_number, umbrella_dist, umbrella_strength, tail_bead, water_bead):
    for i in range(0,replicas):
        
        #### Create new folder
        path = f"./umbrella-run-dir/{str(int(umbrella_window_number))}.{str(i)}"
        
        os.mkdir(path,755)
        subprocess.call(f'chmod 755 {path}', shell = True)
        
        #### Choose the correct frame for the run
        descent_trj = md.Universe (f'{build_dir}/eq1.gro',f'{current_dir}/run_0{str(i)}/traj_comp.xtc')
        cv_dat = f'{current_dir}/run_0{str(i)}/pullx.xvg'
        cv = np.loadtxt(cv_dat, usecols=1, skiprows=17)
        tgtframe = int(((np.abs(cv-umbrella_dist).argmin()*2)/4000)) ## finds the closest number and returns its index
        descent_trj.trajectory[tgtframe]
        descent_trj.atoms.write(f'{path}/start.gro')
        
        #### Edit .mdp files
        with open('common_files/md_template_1us.mdp', 'r') as f:
            ogfile= f.read()
        newfile=ogfile.replace('XXX', str(umbrella_dist))
        newfile=newfile.replace('YYY', str(umbrella_strength))
        newfile=newfile.replace('TTT', str(tail_bead))
        newfile=newfile.replace('WWW', str(water_bead))
        with open(f'{path}/production.mdp', 'w') as f:
            f.write(newfile)

        #### Run the GROMPP
        ### Prepare the simulation
        subprocess.call(f"gmx grompp -f {path}/production.mdp -c {path}/start.gro -r {build_dir}/restraint_reference.gro \
        -p {build_dir}/system_W.top -n {build_dir}/index.ndx -o {path}/topol.tpr -po {path}/mdout.mdp"
                        , shell = True)

####  Generate windows
============================================================

In [8]:
#### Extract 19 starting positions for following PMF
windows = np.array([[0, 0.10, 5000.0],
                    [1, 0.15, 5000.0],
                    [2, 0.20, 5000.0],
                    [3, 0.25, 5000.0],
                    [4, 0.30, 5000.0],
                    [5, 0.35, 5000.0],
                    [6, 0.40, 5000.0],
                    [7, 0.45, 5000.0],
                    [8, 0.50, 5000.0],
                    [9, 0.55, 5000.0],
                    [10, 0.60, 5000.0],
                    [11, 0.65, 5000.0],
                    [12, 0.70, 5000.0],
                    [13, 0.75, 5000.0],
                    [14, 0.80, 5000.0],
                    [15, 0.85, 5000.0],
                    [16, 0.90, 5000.0],
                    [17, 0.95, 5000.0],
                    [18, 1.00, 5000.0]])

replicas = 5

In [9]:
!rm -r umbrella-run-dir
!mkdir -p umbrella-run-dir

In [10]:
for i in windows:
    create_window(i[0],i[1],i[2], tail_bead, water_bead)

/localSSD/mvalerio/virtualenvironments/lib/python3.11/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/localSSD/mvalerio/virtualenvironments/lib/python3.11/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: D
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/localSSD/mvalerio/virtualenvironments/lib/python3.11/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: G
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/localSSD/mvalerio/virtualenvironments/lib/python3.11/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: R
  warnings.warn

Setting the LD random seed to -67127947

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'POPC'

Excluding 1 bonded neighbours molecule type 'PAPS'

Excluding 1 bonded neighbours molecule type 'PAPC'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'DIPE'

Excluding 1 bonded neighbours molecule type 'DPSM'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'POPC'

Excluding 1 bonded neighbours molecule type 'PAPC'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'DIPE'

Excluding 1 bonded neighbours molecule type 'DPSM'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'PAPS'

Excluding 1 bonded neighbours molecule type 'SAP6'

Excluding 1 bonded neighbours molecule type 'MC3H'

Excluding 1 bonded neighbours molecule type 'DPPC'

Excluding 1 

      :-) GROMACS - gmx grompp, 2021.6-dev-20240107-fd1512c96-unknown (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff     

Setting the LD random seed to -1081164385

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'POPC'

Excluding 1 bonded neighbours molecule type 'PAPS'

Excluding 1 bonded neighbours molecule type 'PAPC'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'DIPE'

Excluding 1 bonded neighbours molecule type 'DPSM'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'POPC'

Excluding 1 bonded neighbours molecule type 'PAPC'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'DIPE'

Excluding 1 bonded neighbours molecule type 'DPSM'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'PAPS'

Excluding 1 bonded neighbours molecule type 'SAP6'

Excluding 1 bonded neighbours molecule type 'MC3H'

Excluding 1 bonded neighbours molecule type 'DPPC'

Excluding 

      :-) GROMACS - gmx grompp, 2021.6-dev-20240107-fd1512c96-unknown (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff     

...

      :-) GROMACS - gmx grompp, 2021.6-dev-20240107-fd1512c96-unknown (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff     

Setting the LD random seed to 884930239

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'POPC'

Excluding 1 bonded neighbours molecule type 'PAPS'

Excluding 1 bonded neighbours molecule type 'PAPC'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'DIPE'

Excluding 1 bonded neighbours molecule type 'DPSM'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'POPC'

Excluding 1 bonded neighbours molecule type 'PAPC'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'DIPE'

Excluding 1 bonded neighbours molecule type 'DPSM'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'PAPS'

Excluding 1 bonded neighbours molecule type 'SAP6'

Excluding 1 bonded neighbours molecule type 'MC3H'

Excluding 1 bonded neighbours molecule type 'DPPC'

Excluding 1 

      :-) GROMACS - gmx grompp, 2021.6-dev-20240107-fd1512c96-unknown (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff     

Setting the LD random seed to -1116016769

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'POPC'

Excluding 1 bonded neighbours molecule type 'PAPS'

Excluding 1 bonded neighbours molecule type 'PAPC'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'DIPE'

Excluding 1 bonded neighbours molecule type 'DPSM'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'POPC'

Excluding 1 bonded neighbours molecule type 'PAPC'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'DIPE'

Excluding 1 bonded neighbours molecule type 'DPSM'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'PAPS'

Excluding 1 bonded neighbours molecule type 'SAP6'

Excluding 1 bonded neighbours molecule type 'MC3H'

Excluding 1 bonded neighbours molecule type 'DPPC'

Excluding 

Pull group 1 'MiddleWater' has 2025 atoms
Pull group 2 'tails' has 3521 atoms
Number of degrees of freedom in T-Coupling group LowerMembrane is 9464.34
Number of degrees of freedom in T-Coupling group UpperMembrane is 8405.41
Number of degrees of freedom in T-Coupling group MiddleWater is 6074.57
Number of degrees of freedom in T-Coupling group OuterWater is 18784.68

NOTE 1 [file ./umbrella-run-dir/18.4/production.mdp]:
  There are 1125 non-linear virtual site constructions. Their contribution
  to the energy error is approximated. In most cases this does not affect
  the error significantly.


NOTE 2 [file ./umbrella-run-dir/18.4/production.mdp]:
  You are using a plain Coulomb cut-off, which might produce artifacts.
  You might want to consider using PME electrostatics.


NOTE: Found pull coords with slice pulling - do not check whether pull groups obey PBC.

Pull group  natoms  pbc atom  distance at start  reference at t=0
       1      2025      8305
       2      3521      6526  